# Unhappy Couple Flight

## Problem

A small airplane has $4$ rows of seats with $3$ seats in each row. Seven passengers have boarded the plane and are distributed randomly among the seats. A married couple is next to board. 

**Q1:**
What is the probability there will be 2 adjacent seats in the same row for the couple?

**Q2:**
Suppose the couple just had a fight and one particular partner (mention no names) does not want to sit beside the other (and be scolded for the entire flight, hypothetically speaking). So they (see! gender neutral pronouns are useful!) pay for a few friends (who will be allocated random seats) to also take the flight, in the hope that the couple will be forced to sit apart.<br />
If they want this probability to be smaller than 25% (it was a bad fight) what is the fewest number of friends that will get a free flight? 

---


## Solution

### Setup

### Development Notes


### Q1

### Q2 